# Twitter Sentiment Analysis
Ming Fong

Linguistics 55AC

Useful: https://medium.com/swlh/coronavirus-python-tutorial-1-520cc960aac1

In [54]:
import tweepy #https://github.com/tweepy/tweepy
import csv
from secrets import *
import pandas as pd
import numpy as np
import re
from textblob import TextBlob

In [2]:
consumer_key = api_key
consumer_secret = api_secret
access_key = access_token
access_secret = access_secret

In [6]:
def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []  
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name, count = 200)
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print(f"getting tweets before {oldest}")
        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name, count=200, max_id = oldest)
        
        #save most recent tweets
        alltweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        print(f"...{len(alltweets)} tweets downloaded so far")
    
    #transform the tweepy tweets into a 2D array that will populate the csv 
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text] for tweet in alltweets]
    
    #write the csv  
    # with open(f'data/{screen_name}_tweets.csv', 'w') as f:
    #     writer = csv.writer(f)
    #     writer.writerow(["id","created_at","text"])
    #     writer.writerows(outtweets)
    
    return outtweets


In [73]:
df = pd.DataFrame(get_all_tweets("realDonaldTrump"))

getting tweets before 1334858997367771136
...400 tweets downloaded so far
getting tweets before 1331757781658034177
...599 tweets downloaded so far
getting tweets before 1329064787142172672
...797 tweets downloaded so far
getting tweets before 1326910477587374080
...997 tweets downloaded so far
getting tweets before 1324061986779504641
...1197 tweets downloaded so far
getting tweets before 1322656588285407233
...1397 tweets downloaded so far
getting tweets before 1321231434271592455
...1597 tweets downloaded so far
getting tweets before 1320163234926764031
...1797 tweets downloaded so far
getting tweets before 1318205801555787780
...1995 tweets downloaded so far
getting tweets before 1316669409616965631
...2192 tweets downloaded so far
getting tweets before 1315659918469402623
...2392 tweets downloaded so far
getting tweets before 1313832209317494785
...2590 tweets downloaded so far
getting tweets before 1311454419838525441
...2790 tweets downloaded so far
getting tweets before 1309624

In [75]:
# Remove links
df[2] = [re.sub(r'http\S+', '', str(x)) for x in df[2]]
# Remove Retweets
df = df[~df[2].str.startswith("RT @")]
df[2] = df[2].replace("", np.nan)
df = df.dropna(subset = [2])